In [2]:
import qutip
import copy
import numpy
import cmath
import gmpy2
from qutip import *
from cmath import *
from numpy import *
from gmpy2 import *

c:\users\kowkat\appdata\local\continuum\miniconda3\envs\qutip\lib\site-packages\qutip\__init__.py:161: UserWarning: matplotlib not found: Graphics will not work.
  warnings.warn("matplotlib not found: Graphics will not work.")


In [3]:
#Gate dictionary for calculating circuits
#Includes Clifford + T gates 

#we use 128 bit precision

with gmpy2.ieee(128) as ctx:
    X=matrix([[0, 1], [1, 0]])
    Z=matrix([[1,0],[0,-1]])
    Y=1j*dot(X, Z)
    H=1/sqrt(2)*matrix([[1, 1], [1, -1]])
    T=matrix([[1, 0],[0, exp(pi/4*1j)]])
    P=matrix([[1, 0],[0, exp(pi/2*1j)]])
    dT=matrix([[1, 0],[0, exp(-pi/4*1j)]])
    rz32=eye(2)*cos(pi/32)-1j*sin(pi/32)*Z
    rz32d=eye(2)*cos(pi/32)+1j*sin(pi/32)*Z
    rz16=eye(2)*cos(pi/16)-1j*sin(pi/16)*Z
    rz16d=eye(2)*cos(pi/16)+1j*sin(pi/16)*Z  
    rz64=eye(2)*cos(pi/64)-1j*sin(pi/64)*Z
    rz64d=eye(2)*cos(pi/64)+1j*sin(pi/64)*Z
    rz256=eye(2)*cos(pi/256)-1j*sin(pi/256)*Z
    rz256d=eye(2)*cos(pi/256)+1j*sin(pi/256)*Z 
    dP=matrix([[1, 0],[0, exp(-pi/2*1j)]])
    Id=eye(2)
    I=Id
dict={'X ':X, 'Z ': Z, 'T ':T, 'P ':P, 'T* ':dT, 'H ':H,'P* ':dP, 'Id ':Id}

In [6]:
# In the current directory we place the files with generated circuits: 
# Here we have files for the rz pi/16 rotations containing '32' in their name
# We store their names in a list for further use

data=[]
import glob
for file in glob.glob("*32*.txt"):
    data.append(file)
    

'32_3.txt'

In [11]:
# For each file we carry out circuit calculation and estimate the errors in unitaries as well as in observables

for file in data:
    f=open(file, 'r')
    l=f.readlines()
    n_l=l[l.index('BEGIN RZ1d32(1)\n')+1:l.index('END RZ1d32\n')]
    n_l=[s.strip('1\n') for s in n_l]
    circ=[]
    U1=1
    for i in n_l:
        circ.append(dict[i])
    
    for i in range(len(circ)):
        U1=dot(U1,circ[i])    
    
    dis=[]
    dis_d=[]
    
    #we want to repeat the calculations with different hermitian operators
    for j in range(100):
        oper=rand_herm(2)

        #we want to compare diagonal operators and any operator: the squared error is only observed in diagonal operators (???)
        #we search for a diagonal hermitian operator
        oper_d=rand_herm(2)
        while (oper_d.data[0,1]!=0):
            while(oper_d.data[1,0]!=0):
                operd=rand_herm(2)
                
        op=matrix([[oper.data[0,0],oper.data[0,1]], [oper.data[1,0], oper.data[1,1]]])
        op_d=matrix([[oper_d.data[0,0],oper_d.data[0,1]], [oper_d.data[1,0], oper_d.data[1,1]]])
        
        #we generate basis state: we want to consider ground states only
        qubit=basis(2,0)
        qubitin=qubit
        nq=array([[qubit.data[0,0]],[qubit.data[1,0]]])
        nqi=nq
        
        #we apply approximating circuit to the 0 ket
        nq=array(U1*nqi)
        knq=nq.transpose()
        knq=[real(knq[0,0])-imag(knq[0,0])*1j,real(knq[0,1])-imag(knq[0,1])*1j]

        #we apply exact rotation to the 0 ket
        qbf=rz32*nqi
        kqbf=qbf.transpose()
        kqbf=[real(kqbf[0,0])-imag(kqbf[0,0])*1j,real(kqbf[0,1])-imag(kqbf[0,1])*1j]
        
        # we calculate the expectation values for four cases: the diagonal operator with exact state and approximated state
        # and the non diagonal operator with exact state and the approximated state 
        e_es_d=knq*op_d*nq
        e_ex_d=kqbf*op_d*qbf
        err_d=abs(e_es_d-e_ex_d)
        
        
        e_es=knq*op*nq
        e_ex=kqbf*op*qbf
        err=abs(e_es-e_ex)
        
        #we keep the errors of each iteration in a table
        dis.append(err)
        dis_d.append(err_d)
        
    # we average the error over 100 measurments
    erravg=sum(dis)/len(dis)
    erravg_d=sum(disd)/len(disd)
        
    a=abs(trace(U1*rz32d))
    d=sqrt(1-a/2)
    print(str(d)+' '+str(erravg_d)+' '+ str(erravg))

0.000392539543490424445159221163439033227 1.27365082125914138085245484607252738e-07 0.000195339823142446115202583847771647273
1.34267110600872162247175571278073505e-05 1.68041892425364525168270230346513546e-10 6.71056306417031389806232675494542105e-06
6.39711897782148903320102511150478673e-07 4.50269346472503099564532758904599814e-13 3.28198152312072429814368097174098474e-07
2.71863804872610063892725865895747644e-10 8.75587707931622240282658412331304799e-20 1.59336785874802385037812536934475293e-10
2.31943777569677956257282685394644186e-14 6.87176671927508858192523290065647071e-28 1.27924104914960927036002077557258396e-14
